In [1]:
from random import random, seed, randrange
from math import floor, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
import pandas as pd

In [32]:
def load_co( Nco, s_box, N_max ):
    
    Xco = [0]*N_max; Yco =[0]*N_max; Zco =[0]*N_max;
    for i in range(Nco ):
        Xco[i] = random()*s_box #[A]
        Yco[i] = random()*s_box #[A]
        Zco[i] = random()*s_box #[A]
    return Xco, Yco, Zco   

def load_me( Nme, s_box, N_max ):
    
    Xme = [0]*N_max; Yme =[0]*N_max; Zme =[0]*N_max;
    for i in range( Nme ):
        Xme[i] = random()*s_box #[A]
        Yme[i] = random()*s_box #[A]
        Zme[i] = random()*s_box #[A]
    return Xme, Yme, Zme   



def move(spec, o, x, y, z):
    # Carry out the "move" trial move
    if( spec == "co2"): 
        Xco[o] = x
        Yco[o] = y
        Zco[o] = z
        
    elif (spec == "me"):
        Xme[o] = x
        Yme[o] = y
        Zme[o] = z
    
    
def add(spec, x, y, z):
    # Carry out the "add" trial move
    global Nco, Nme
    if( spec == "co2"): 
        Xco[Nco] = x
        Yco[Nco] = y
        Zco[Nco] = z
        Nco = Nco + 1
        
    elif (spec == "me"):
        Xme[Nme] = x
        Yme[Nme] = y
        Zme[Nme] = z
        Nme = Nme + 1
    
    
def remove(spec, o):
    # Carry out the "remove" trial move
    global Nco, Nme
    if( spec == "co2"): 
        Nco = Nco - 1
        Xco[o] = Xco[Nco]
        Yco[o] = Yco[Nco]
        Zco[o] = Zco[Nco]
        
    elif (spec == "me"):
        Nme = Nme - 1
        Xme[o] = Xme[Nme]
        Yme[o] = Yme[Nme]
        Zme[o] = Zme[Nme]
        
    


def dist_hi(spec,x,y,z,j):
    # Distance btw a proposed particle and the ith H2 particle
    if(spec == "co2"):
        dx = x - Xco[j] #[A]
        dy = y - Yco[j] #[A]
        dz = z - Zco[j] #[A]
    elif (spec == "me"):
        dx = x - Xme[j] #[A]
        dy = y - Yme[j] #[A]
        dz = z - Zme[j] #[A]

    if (dx > 0.5*s_box):
        dx = dx-s_box 
    elif (dx < -0.5*s_box):
        dx = dx + s_box
    if (dy > 0.5*s_box):
        dy = dy-s_box
    elif (dy < -0.5*s_box):
        dy = dy + s_box
    if (dz > 0.5*s_box):
        dz = dz-s_box
    elif (dz < -0.5*s_box):
        dz = dz + s_box
    return dx*dx + dy*dy + dz*dz

def dist_ci(x,y,z,j):
    # Distance btw proposed particle and the ith C particle
    dx = x - Xc[j] #[A]
    dy = y - Yc[j] #[A]
    dz = z - Zc[j] #[A]
    if (dx > 0.5*s_box):
        dx = dx-s_box
    elif (dx < -0.5*s_box):
        dx = dx + s_box
    if (dy > 0.5*s_box):
        dy = dy-s_box
    elif (dy < -0.5*s_box):
        dy = dy + s_box
    if (dz > 0.5*s_box):
        dz = dz-s_box
    elif (dz < -0.5*s_box):
        dz = dz + s_box
    return dx*dx + dy*dy + dz*dz



def Ui(r2, eps, sig):
    # LJ potential btw two particles at r distance away
    if ( r2 < rc*rc ):
        r2i = sig*sig/r2 #[]
        r6i = r2i*r2i*r2i
        En = 4*eps*(r6i*r6i-r6i) #[K]
        Vir = 48*eps*(r6i*r6i-0.5*r6i) #[K]
    else:
        En = 0
        Vir = 0
    return En, Vir

def Up(spec, x, y, z, ia, jb=0):
    # Total LJ potential of proposed particle with all other particles
    # omit the ia'th H2 particle. When not needed, Nh is used.
    # jb used in the UTo operation to avoid overcounting interactions
    En_move = 0
    Vir_move = 0
    
    if(spec == "co2"):
        #Similar Particles
        for i in range(jb, Nco): # H2 particles
            if ( i != ia):
                r2 = dist_hi("co2", x,y,z, i)
                ui, viri = Ui( r2, e_co, s_co)
                En_move = En_move + ui
                Vir_move = Vir_move + viri
        #Other Mobile
        
        for i2 in range(0,Nme):
            r2 = dist_hi("me", x,y,z, i)
            ui, viri = Ui( r2, e_meco, s_meco)
            En_move = En_move + ui
            Vir_move = Vir_move + viri
        
        for ic in range(Nc): # C particles
            r2 = dist_ci( x,y,z, ic)
            ui, viri = Ui( r2, e_cco, s_cco)
            En_move = En_move + ui
            Vir_move = Vir_move + viri
        
    elif (spec == "me"):
        #Similar Particles
        for i in range(jb, Nme): # H2 particles
            if ( i != ia):
                r2 = dist_hi("me", x,y,z, i)
                ui, viri = Ui( r2, e_me, s_me)
                En_move = En_move + ui
                Vir_move = Vir_move + viri
        #Other Mobile
        
        for i2 in range(0,Nco):
            r2 = dist_hi("co2", x,y,z, i)
            ui, viri = Ui( r2, e_meco, s_meco)
            En_move = En_move + ui
            Vir_move = Vir_move + viri
        
        for ic in range(Nc): # C particles
            r2 = dist_ci( x,y,z, ic)
            ui, viri = Ui( r2, e_cme, s_cme)
            En_move = En_move + ui
            Vir_move = Vir_move + viri
                 
    return En_move, Vir_move

In [17]:
e_me = 147.9 # eps over kb[K]
s_me = 3.73 # sigma [A]

e_co = 309
s_co = 3.36

e_c = 28.0 #Activated is 89.44
s_c = 3.4

e_meco = np.sqrt(e_me*e_co)
s_meco = (s_me+s_co)/2

e_cme = np.sqrt(e_me*e_c)
s_cme = (s_me+s_c)/2

e_cco = np.sqrt(e_co*e_c)
s_cco = (s_co + s_c)/2

y_co = 0.5
P_res = 1000000 #Pa
T = 77 #K
s_box = 20
N_max = 1000
Nco = 10
Nme = 10

In [18]:
Xco, Yco, Zco = load_co(Nco, s_box, N_max)
Xme, Yme, Zme = load_me(Nme, s_box, N_max)

In [33]:
x = random()*s_box
y = random()*s_box
z = random()*s_box
print("Coords", x, y, z)
add("me", x, y, z )
print( Xme[1:Nme] )

Coords 18.288561848851298 9.051527604367442 2.055877811981215
[8.969027499662667, 15.986482559873343, 6.016706523589259, 1.3579882019678324, 18.35289489291623, 2.6789074346788078, 7.05934817208904, 3.4528671048803283, 15.592540694834245, 7.131569650680378, 3.6910421971418805, 13.959905635919, 6.248517058334116, 12.375656224637634, 18.288561848851298]


In [40]:
o = floor( random()*Nco )
print("index", o )
remove("co2", o)
print( Xco[1:Nco])

index 0
[1.4089285348239966, 5.737698583942117, 3.7381063955488036, 12.934924514771108, 4.6646627854418865, 11.864400843307745, 13.767280024416351, 0.0683753116466268, 3.553507715214137]


In [63]:
o = floor( random()*Nco )
print("index", o )
x = random()*s_box
y = random()*s_box
z = random()*s_box
print("Coords", x, y, z)
move("co2", o, x, y, z)
print( Xco[1:Nco] )

index 1
Coords 1.3038361047419689 15.201647957326646 5.795438403311675
[1.3038361047419689, 13.457003016660774, 5.521336524625315, 12.934924514771108, 1.5219005742361302, 7.084074709905186, 6.6011246284725145, 19.448395558247395, 8.931047177136346]
